## Optional Challenge

Try your lane finding pipeline on the video below.  Does it still work?  Can you figure out a way to make it more robust?  If you're up for the challenge, modify your pipeline so it works with this video and submit it along with the rest of your project!

In [2]:
import math
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import cv2
from moviepy.editor import VideoFileClip
from IPython.display import HTML
%matplotlib inline

def grayscale(img):
    return cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    # Or use BGR2GRAY if you read an image with cv2.imread()
    # return cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    
def canny(img, low_threshold, high_threshold):
    """Applies the Canny transform"""
    return cv2.Canny(img, low_threshold, high_threshold)

def gaussian_blur(img, kernel_size):
    return cv2.GaussianBlur(img, (kernel_size, kernel_size), 0)

def region_of_interest(img, vertices):
    #defining a blank mask to start with
    mask = np.zeros_like(img)   
    
    #defining a 3 channel or 1 channel color to fill the mask with depending on the input image
    if len(img.shape) > 2:
        channel_count = img.shape[2]  # i.e. 3 or 4 depending on your image
        ignore_mask_color = (255,) * channel_count
    else:
        ignore_mask_color = 255
        
    #filling pixels inside the polygon defined by "vertices" with the fill color    
    cv2.fillPoly(mask, vertices, ignore_mask_color)
    
    #returning the image only where mask pixels are nonzero
    masked_image = cv2.bitwise_and(img, mask)
    return masked_image


def draw_lines(img, lines, color=[255, 0, 0], thickness=2):
    for line in lines:
        for x1,y1,x2,y2 in line:
            cv2.line(img, (x1, y1), (x2, y2), color, thickness)

def hough_lines(img, rho, theta, threshold, min_line_len, max_line_gap):
    lines = cv2.HoughLinesP(img, rho, theta, threshold, np.array([]), 
                            minLineLength=min_line_len, maxLineGap=max_line_gap)
    line_img = np.zeros((img.shape[0], img.shape[1], 3), dtype=np.uint8)
    draw_lines(line_img, lines)
    return line_img

# Python 3 has support for cool math symbols.

def weighted_img(img, initial_img, α=0.8, β=1., λ=0.):
    return cv2.addWeighted(initial_img, α, img, β, λ)

def line_length(lines):
    length = np.sqrt((lines[:, 1] - lines[:, 3]) ** 2 + \
                     (lines[:, 0] - lines[:, 2]) ** 2)
    return length


def top_average(lines, threshold):
    idx = lines[:, 1] < threshold
    
    try:
        lines = lines[idx]
    except IndexError:
        # print('in exception', lines[:, 1], threshold, idx)
        return np.array([0, 0], dtype=np.int32)
        
    tops = lines[:, :2]
    
    lengths = line_length(lines)
    lengths = lengths / lengths.sum()
    lengths = np.expand_dims(lengths, axis=-1)
        
    mean_top = (tops * lengths).sum(axis=0)

    return mean_top


def infer_bottom(top, slop):
    bottom_y = 539
    bottom_x = (bottom_y - top[1]) / slop + top[0]
    return np.array([bottom_x, bottom_y])

def combine_line(top, bottom):
    line = np.concatenate([top, bottom])
    line = np.around(line).astype(np.uint32)
    line[0] = np.clip(line[0], 0, 959)
    line[2] = np.clip(line[2], 0, 959)
    line[1] = np.clip(line[1], 0, 539)
    line[3] = np.clip(line[3], 0, 539)
    return line
    
def extrapolate_lane(lines):
    
    slop_upper = 0.45
    slop_lower = -0.45
    
    origin = lines
    lines = np.squeeze(lines)
    slops = (lines[:, 1] - lines[:, 3]) / (lines[:, 0] - lines[:, 2])
    slops = np.squeeze(slops)
    
    pos_lines = lines[slops > slop_upper]
    neg_lines = lines[slops < slop_lower]
    
    pos_length = line_length(pos_lines)
    pos_length = pos_length / pos_length.sum()
    
    neg_length = line_length(neg_lines)
    neg_length = neg_length / neg_length.sum()
        
    average_pos_slop = (slops[slops > slop_upper] * pos_length).sum()
    average_neg_slop = (slops[slops < slop_lower] * neg_length).sum()
    
    # print(average_pos_slop, average_neg_slop)
    
    pos_top = top_average(pos_lines, 370)
    
    # exception case
    if pos_top[1] < 10:
        pos_top = top_average(pos_lines, 539)
    # print('pos_top:', pos_top)
    
    pos_bottom = infer_bottom(pos_top, average_pos_slop)
    
    neg_top = top_average(neg_lines, 400)
    
    # exception case
    if neg_top[1] < 10:
        neg_top = top_average(neg_lines, 539)
    # print ('neg_top', neg_top, neg_lines)
    
    neg_bottom = infer_bottom(neg_top, average_neg_slop)
    
    pos = combine_line(pos_top, pos_bottom)
    neg = combine_line(neg_top, neg_bottom)
    ret = np.vstack((pos, neg))
    ret = ret.reshape(-1, 1, 4)
    return ret

def process_image_challenge(image):
    # NOTE: The output you return should be a color image (3 channel) for processing video below
    # TODO: put your pipeline here,
    # you should return the final output (image where lines are drawn on lanes)
    image = cv2.resize(image, (960, 540))
    gray_image = grayscale(image)
    blur = gaussian_blur(gray_image, kernel_size=3)

    edge = canny(blur, 50, 150)

    vertices = np.array([[(150, 539), (890, 539), (465, 295), (485, 295)]], dtype=np.int32)

    interest_edge = region_of_interest(edge, vertices)

    rho = 1 # distance resolution in pixels of the Hough grid
    theta = 1*np.pi/180 # angular resolution in radians of the Hough grid
    threshold = 30     # minimum number of votes (intersections in Hough grid cell)
    min_line_length = 5  #minimum number of pixels making up a line
    max_line_gap = 2    # maximum gap in pixels between connectable line segments

    lines = cv2.HoughLinesP(interest_edge, rho, theta, threshold, np.array([]), 
                            minLineLength=min_line_length, maxLineGap=max_line_gap)

    extrapolate = extrapolate_lane(lines)
    hough_image = np.zeros((image.shape[0], image.shape[1], 3), dtype=np.uint8)

    draw_lines(hough_image, extrapolate, thickness=5)
    marked_img = weighted_img(hough_image, image)
    return marked_img

In [3]:
challenge_output = 'test_videos_output/challenge.mp4'
## To speed up the testing process you may want to try your pipeline on a shorter subclip of the video
## To do so add .subclip(start_second,end_second) to the end of the line below
## Where start_second and end_second are integer values representing the start and end of the subclip
## You may also uncomment the following line for a subclip of the first 5 seconds
##clip3 = VideoFileClip('test_videos/challenge.mp4').subclip(0,5)
clip3 = VideoFileClip('test_videos/challenge.mp4')
challenge_clip = clip3.fl_image(process_image_challenge)
%time challenge_clip.write_videofile(challenge_output, audio=False)

[MoviePy] >>>> Building video test_videos_output/challenge.mp4
[MoviePy] Writing video test_videos_output/challenge.mp4


100%|██████████| 251/251 [00:05<00:00, 49.51it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: test_videos_output/challenge.mp4 

CPU times: user 13.7 s, sys: 316 ms, total: 14 s
Wall time: 5.44 s


In [4]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(challenge_output))